In [4]:
pip install numpy pandas sklearn nltk pyvi networkx gensim joblib

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\toan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# read file contain stopwords
def read_vi_stopwords(file_name):
  file = open(file_name, encoding="utf8")
  filedata = file.read()
  stop_words=[]
  stop_words = [stopword.rstrip('\n') for stopword in open(file_name, encoding="utf8")]
  return stop_words

In [7]:
# function to remove stopwords
stop_words=read_vi_stopwords(r'C:\Users\toan\Desktop\KHOA LUAN\Sprint 10\vietnamese-stopwords.txt')
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [9]:
word_embeddings = {}
f = open(r'C:\Users\toan\Desktop\KHOA LUAN\Sprint 10\vi.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
def textRank(content):
    contents_parsed = content.lower() #Biến đổi hết thành chữ thường
    contents_parsed = contents_parsed.replace('.\n', '. ')
    contents_parsed = contents_parsed.replace('\n', '. ') # Đổi các ký tự xuống dòng thành chấm câu 
    contents_parsed = contents_parsed.strip() #Loại bỏ đi các khoảng trắng thừa
    import nltk
    nltk.download('punkt')
    origin_sentences = nltk.sent_tokenize(contents_parsed)
    sentences_without_sw = [remove_stopwords(r.split()) for r in origin_sentences]
    sentence_vectors = []
    for i in sentences_without_sw:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)
    # similarity matrix
    sim_mat = np.zeros([len(sentences_without_sw), len(sentences_without_sw)])
    from sklearn.metrics.pairwise import cosine_similarity
    # khởi tạo ma trận với điểm số tương tự cosine.
    for i in range(len(sentences_without_sw)):
      for j in range(len(sentences_without_sw)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    # chuyển đổi ma trận tương tự sim_mat thành một đồ thị
    import networkx as nx
    nx_graph = nx.from_numpy_array(sim_mat)
    # áp dụng thuật toán Xếp hạng trang để xếp hạng câu
    scores = nx.pagerank(nx_graph)
    import heapq
    top_sentences = heapq.nlargest(2, scores, key=scores.get)
    summary=''
    for i in top_sentences:
      summary+=origin_sentences[i]
    return summary

In [11]:
import joblib
joblib.dump(word_embeddings, "./word_embeddings.joblib", compress=True)
joblib.dump(textRank, "./textRank.joblib", compress=True)

['./textRank.joblib']